# Develon

In [ ]:
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urlparse
import pandas as pd
import openpyxl
from tqdm import tqdm  # For progress bar

# Function to extract domain from URL
def extract_domain(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

# Function to perform a Google search and return the driver
def search_google(query):
    search_url = f"https://www.google.com/search?q={query}"
    
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("start-maximized")
    options.add_argument(
        f"user-agent=Mozilla/5.0 (Windows NT {random.randint(6, 10)}.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random.randint(90, 110)}.0.4472.{random.randint(100, 200)} Safari/537.36"
    )
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(search_url)
    return driver

# Function to extract details table from the first search result
def get_details_from_result(url, model):
    query = extract_domain(url) + "+" + model
    driver = search_google(query)
    
    try:
        # Find the first search result
        first_result = driver.find_element(By.CSS_SELECTOR, 'a[jsname="UWckNb"]')
        link = first_result.get_attribute('href')  # Extract the href attribute (URL)
        driver.get(link)
        
        # Extract the table from the details page
        try:
            detail_table = driver.find_element(By.CSS_SELECTOR, ".equipment-detail_spec-table.excavators1.en_plain_l")
            rows = detail_table.find_elements(By.TAG_NAME, "tr")
            details = []
            for row in rows[1:]:
                cells = row.find_elements(By.TAG_NAME, "th")
                if len(cells) > 1:  # Check if there is a second column
                    details.append(cells[1].text)  # Extract the second column's text
        except NoSuchElementException:
            details = ["Details not found"]
    except Exception as e:
        details = [f"Error: {e}"]
    finally:
        driver.quit()
    
    return details

# Main function to scrape excavator data
def scrape_excavator_data():
    url = "https://asia.develon-ce.com/en/equipment/excavators/?param1=CIS%2COCEANIA%2CSOUTH_EAST_ASIA%2CINDIA&param2=&param3="
    
    options = webdriver.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)

    # Create an Excel workbook
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Excavator Data"
    sheet.append(["Model"] + [f"Detail {i}" for i in range(1, 21)])  # Adjust number of details as needed

    # Track already scraped models
    scraped_models = set()

    while True:
        try:
            model_elements = driver.find_elements(By.CSS_SELECTOR, ".product_name.en_plain_b")
            
            for model_element in tqdm(model_elements, total=len(model_elements), desc="Scraping Models"):
                model_name = model_element.text
                
                # Skip if model already scraped
                if model_name in scraped_models:
                    continue

                # Get details from search result
                details = get_details_from_result(url, model_name)
                sheet.append([model_name] + details)

                # Mark the model as scraped
                scraped_models.add(model_name)
            
            # Handle "Load More" button if available
            try:
                load_more_button = driver.find_element(By.CLASS_NAME, "btn_more")
                ActionChains(driver).move_to_element(load_more_button).perform()
                load_more_button.click()
            except NoSuchElementException:
                print("No 'Load More' button found. Exiting loop.")
                break
        except Exception as e:
            print(f"Error: {e}")
            break

    # Save data to an Excel file
    file_name = "develon_excavator_data_combined.xlsx"
    workbook.save(file_name)
    print(f"Data successfully saved to {file_name}")

    driver.quit()

if __name__ == "__main__":
    scrape_excavator_data()


Error: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=132.0.6834.84)
Stacktrace:
0   chromedriver                        0x0000000102bc37a4 cxxbridge1$str$ptr + 2589716
1   chromedriver                        0x0000000102bbc05c cxxbridge1$str$ptr + 2559180
2   chromedriver                        0x000000010275ff5c cxxbridge1$string$len + 88260
3   chromedriver                        0x000000010273b87c chromedriver + 129148
4   chromedriver                        0x00000001027cb91c cxxbridge1$string$len + 529028
5   chromedriver                        0x00000001027de800 cxxbridge1$string$len + 606568
6   chromedriver                        0x0000000102798e40 cxxbridge1$string$len + 321448
7   chromedriver                        0x0000000102799a88 cxxbridge1$string$len + 324592
8   chromedriver                        0x0000000102b8e8a0 cxxbridge1$str$ptr + 2372880
9   chromedriver                        0x0000000102b91

# Develon - Prototype

### Currently disable google search

In [ ]:
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urlparse
import pandas as pd
import openpyxl
from tqdm import tqdm  # For progress bar

# Function to extract domain from URL
def extract_domain(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

# Function to perform a Google search and return the driver
def search_google(query):
    search_url = f"https://www.google.com/search?q={query}"
    
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("start-maximized")
    options.add_argument(
        f"user-agent=Mozilla/5.0 (Windows NT {random.randint(6, 10)}.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random.randint(90, 110)}.0.4472.{random.randint(100, 200)} Safari/537.36"
    )
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(search_url)
    return driver

# Function to extract details table from the first search result
def get_details_from_result(url, model):
    query = extract_domain(url) + "+" + model
    driver = search_google(query)
    
    try:
        # Find the first search result
        first_result = driver.find_element(By.CSS_SELECTOR, 'a[jsname="UWckNb"]')
        link = first_result.get_attribute('href')  # Extract the href attribute (URL)
        driver.get(link)
        
        # Extract the table from the details page
        headers = []
        details = []
        try:
            detail_table = driver.find_element(By.CSS_SELECTOR, ".equipment-detail_spec-table.excavators1.en_plain_l")
            rows = detail_table.find_elements(By.TAG_NAME, "tr")
            
            # Skip the first row
            for row in rows[1:]:  # Start iteration from the second row
                cells = row.find_elements(By.TAG_NAME, "th")
                if len(cells) > 1:  # Ensure there are at least two columns
                    headers.append(cells[0].text)  # First column (header)
                    details.append(cells[1].text)  # Second column (detail)
                else:
                    headers.append("")  # Placeholder if header is missing
                    details.append("")  # Placeholder if detail is missing
        except NoSuchElementException:
            headers = ["Header not found"]
            details = ["Details not found"]
    except Exception as e:
        headers = [f"Error: {e}"]
        details = [f"Error: {e}"]
    finally:
        driver.quit()
    
    return headers, details


# Main function to scrape excavator data
def scrape_excavator_data():
    url = "https://asia.develon-ce.com/en/equipment/excavators/?param1=CIS%2COCEANIA%2CSOUTH_EAST_ASIA%2CINDIA&param2=&param3="
    
    options = webdriver.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)

    # Create an Excel workbook
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Excavator Data"

    # Track already scraped models
    scraped_models = set()

    while True:
        try:
            model_elements = driver.find_elements(By.CSS_SELECTOR, ".product_name.en_plain_b")
            
            for model_element in tqdm(model_elements, total=len(model_elements), desc="Scraping Models"):
                model_name = model_element.text
                
                # Skip if model already scraped
                if model_name in scraped_models:
                    continue

                # Get headers and details from search result
                headers, details = get_details_from_result(url, model_name)
                
                # Set column headers only once
                if sheet.max_row == 1:
                    sheet.append(["Model"] + headers)

                # Append model and details
                sheet.append([model_name] + details)

                # Mark the model as scraped
                scraped_models.add(model_name)
            
            # Handle "Load More" button if available
            try:
                load_more_button = driver.find_element(By.CLASS_NAME, "btn_more")
                ActionChains(driver).move_to_element(load_more_button).perform()
                load_more_button.click()
            except NoSuchElementException:
                print("No 'Load More' button found. Exiting loop.")
                break
        except Exception as e:
            print(f"Error: {e}")
            break

    # Save data to an Excel file
    file_name = "develon_excavator_data_combined_prototype.xlsx"
    workbook.save(file_name)
    print(f"Data successfully saved to {file_name}")

    driver.quit()

if __name__ == "__main__":
    scrape_excavator_data()


Scraping Models: 100%|██████████| 45/45 [00:15<00:00,  2.97it/s] 

No 'Load More' button found. Exiting loop.
Data successfully saved to develon_excavator_data_combined(2).xlsx


# Komatsu

In [8]:
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urlparse
import openpyxl
from tqdm import tqdm  # For progress bar


# Function to extract domain from URL
def extract_domain(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc


# Function to perform a Google search and return the driver
def search_google(query):
    search_url = f"https://www.google.com/search?q={query}"
    
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("start-maximized")
    options.add_argument(
        f"user-agent=Mozilla/5.0 (Windows NT {random.randint(6, 10)}.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random.randint(90, 110)}.0.4472.{random.randint(100, 200)} Safari/537.36"
    )
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(search_url)
    return driver


# Function to extract details table from the first search result
def get_details_from_result(url, model):
    query = extract_domain(url) + "+" + model
    driver = search_google(query)
    
    try:
        # Find the first search result
        first_result = driver.find_element(By.CSS_SELECTOR, 'a[jsname="UWckNb"]')
        link = first_result.get_attribute('href')  # Extract the href attribute (URL)
        driver.get(link)
        
        # Extract the table from the details page
        try:
            detail_table = driver.find_element(By.CSS_SELECTOR, "table-wrapper")
            rows = detail_table.find_elements(By.TAG_NAME, "tr")
            details = []
            for row in rows:
                cells = row.find_elements(By.TAG_NAME, "td")
                if len(cells) > 1:  # Check if there is a second column
                    details.append(cells[1].text)  # Extract the second column's text
        except NoSuchElementException:
            details = ["Details not found"]
    except Exception as e:
        details = [f"Error: {e}"]
    finally:
        driver.quit()
    
    return details


# Main function to scrape excavator data
def scrape_excavator_data():
    url = "https://www.komatsu.com/en/products/excavators/"
    
    options = webdriver.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)

    # Create an Excel workbook
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Excavator Data"
    sheet.append(["Model"] + [f"Detail {i}" for i in range(1, 21)])  # Adjust number of details as needed

    # Track already scraped models
    scraped_models = set()

    while True:
        try:
            # Wait for models to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".product-card__name"))
            )
            model_elements = driver.find_elements(By.CSS_SELECTOR, ".product-card__name")
            
            for model_element in tqdm(model_elements, total=len(model_elements), desc="Scraping Models"):
                model_name = model_element.text.strip()
                
                if model_name in scraped_models:
                    continue
                
                # Get details from search result
                details = get_details_from_result(url, model_name)
                sheet.append([model_name] + details)

                # Mark the model as scraped
                scraped_models.add(model_name)
            
            # Handle "Load More" button if available
            try:
                load_more_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".action-button.action-button--secondary.product-tiles__load-more"))
                )
                ActionChains(driver).move_to_element(load_more_button).perform()
                load_more_button.click()
            except TimeoutException:
                print("No 'Load More' button found. Exiting loop.")
                break
        except Exception as e:
            print(f"Error: {e}")
            break

    # Save data to an Excel file
    file_name = "komatsu_excavator_data_combined.xlsx"
    workbook.save(file_name)
    print(f"Data successfully saved to {file_name}")

    driver.quit()


if __name__ == "__main__":
    scrape_excavator_data()


Scraping Models: 100%|██████████| 9/9 [02:52<00:00, 19.19s/it]


Error: Message: element click intercepted: Element <a href="#" data-tag-trigger="click" data-tag-meta="{&quot;type&quot;: &quot;cta&quot;, &quot;category&quot;: &quot;Button&quot;, &quot;action&quot;: &quot;...&quot;, &quot;elementNo&quot;: &quot;&quot;}" class="action-button action-button--secondary product-tiles__load-more">Load more products</a> is not clickable at point (725, 638). Other element would receive the click: <div id="onetrust-button-group-parent" class="ot-sdk-three ot-sdk-columns has-reject-all-button">...</div>
  (Session info: chrome=132.0.6834.84)
Stacktrace:
0   chromedriver                        0x0000000100f737a4 cxxbridge1$str$ptr + 2589716
1   chromedriver                        0x0000000100f6c05c cxxbridge1$str$ptr + 2559180
2   chromedriver                        0x0000000100b0ff5c cxxbridge1$string$len + 88260
3   chromedriver                        0x0000000100b5a954 cxxbridge1$string$len + 393916
4   chromedriver                        0x0000000100b58f30 

# Two-wheeler
